In [3]:
import pandas as pd # import Pandas library
import gzip # to manage gzip-compressed files
import json # to handle json data
from textblob import TextBlob # to make the sentiment analyst

In [2]:
def convert_single_to_double_quotes(line):
    """Safely converts single quotes in a string to double quotes for JSON parsing."""
    try:
        return json.dumps(eval(line)) #eval first the incorrect json format of simple quote, into a python dictionary, since python
                                      #actually uses simple in dictionaries.
                                      #Then, json.dumps converts that string (dictionary) into a json with a real correct format of
                                      #two quotation marks
    except:
        return None

In [3]:
# List to store the converted lines
converted_lines2 = []

# Open the compressed file and read line by line with utf-8 encoding
with gzip.open('./datasets_crudos/users_items.json.gz', 'rt', encoding='utf-8') as file:
    for line in file:
        converted_line = convert_single_to_double_quotes(line)
        if converted_line:
            converted_lines2.append(converted_line)

# Load the converted lines into a DataFrame
users_items = pd.DataFrame([json.loads(line) for line in converted_lines2])

In [12]:
# make explode of column items to keep later only the needed data, and to create a joined_id to join table user_reviews. 
users_items1 = users_items.explode(["items"])

In [27]:
users_items1.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [14]:
users_items2 = pd.json_normalize(users_items1['items']).set_index(users_items1['items'].index)

In [15]:
users_items2.head()

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6.0,0.0
0,20,Team Fortress Classic,0.0,0.0
0,30,Day of Defeat,7.0,0.0
0,40,Deathmatch Classic,0.0,0.0
0,50,Half-Life: Opposing Force,0.0,0.0


In [16]:
users_items3= pd.concat([users_items2, users_items1], axis = 1)

In [28]:
users_items3.head()

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url,items
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."


In [35]:
users_items3.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'user_id', 'items_count', 'steam_id', 'user_url', 'items'],
      dtype='object')

In [36]:
#delete columns playtime_2weeks, user_url, items
users_items4 = users_items3.drop(['playtime_2weeks','steam_id','user_url','items'], axis=1)

In [38]:
# create a joined_id between user_id & item_id to joint user_reviews table with user_items table and steam_games table.
users_items4['joined_id'] = users_items4['user_id'] + users_items4['item_id']

In [39]:
users_items5 = users_items4.drop(['user_id', 'item_id'], axis=1) # drop original id columns

In [43]:
users_items5.dropna(how='all', inplace=True) # the how=all ensures to drop only the rows that are completely empty

In [ ]:
users_items5.iloc[300:350]

In [48]:
print(users_items5.isna().sum()) # check for na values

item_name           16806
playtime_forever    16806
items_count             0
joined_id           16806
dtype: int64


In [47]:
users_items6 = users_items5.dropna(subset=['playtime_forever']) # delete rows where playtime_forever = na

In [51]:
print(users_items6.isna().sum()) # check for na values update

item_name           0
playtime_forever    0
items_count         0
joined_id           0
dtype: int64


In [60]:
users_items7 = users_items6[users_items6['playtime_forever'] != 0] # drop rows where timeplay_forever = 0

In [71]:
# read user_review to join with the user_items
file_name_to_read = './dataset_ready/users_reviews_v2.csv'
users_reviews4 = pd.read_csv(file_name_to_read)

In [72]:
users_reviews4.head()

,item_id,recommend,review,user_id,joined_id
0,1250.0,True,Simple yet with great replayability. In my opi...,76561197970982479,765611979709824791250
1,22200.0,True,It's unique and worth a playthrough.,76561197970982479,7656119797098247922200
2,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,7656119797098247943110
3,251610.0,True,I know what you think when you see this title ...,js41637,js41637251610
4,227300.0,True,For a simple (it's actually not all that simpl...,js41637,js41637227300


In [73]:
merged_df = users_items7.merge(users_reviews4, on='joined_id') # merge items with reviews tables

In [74]:
merged_df.head()

,item_name,playtime_forever,items_count,joined_id,item_id,recommend,review,user_id
0,Zeno Clash,271.0,277,7656119797098247922200,22200.0,True,It's unique and worth a playthrough.,76561197970982479
1,Killing Floor,10006.0,277,765611979709824791250,1250.0,True,Simple yet with great replayability. In my opi...,76561197970982479
2,Metro 2033,834.0,277,7656119797098247943110,43110.0,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,Euro Truck Simulator 2,551.0,888,js41637227300,227300.0,True,For a simple (it's actually not all that simpl...,js41637
4,"Papers, Please",349.0,888,js41637239030,239030.0,True,Very fun little game to play when your bored o...,js41637


In [75]:
# Export DataFrame to csv file to check
file_name = './dataset_ready/merged_users_items_reviews.csv' #file name included extension
pd.DataFrame(merged_df, columns = merged_df.columns).to_csv(file_name, index=False)  # "index=False" to avoid indexs columns

In [86]:
file_name_to_read = './dataset_ready/merged_users_items_reviews.csv' # to load the merge reviews and items file 
merged_df = pd.read_csv(file_name_to_read)

In [90]:
merged_df[merged_df['item_id'] == 22200] # just to check correct file

,item_name,playtime_forever,items_count,joined_id,item_id,recommend,review,user_id
0,Zeno Clash,271.0,277,7656119797098247922200,22200.0,True,It's unique and worth a playthrough.,76561197970982479
16647,Zeno Clash,240.0,942,seantheextraprawnsheepguy22200,22200.0,True,"Combine FPS and Fighting games, mixed with the...",seantheextraprawnsheepguy
22570,Zeno Clash,690.0,61,pipekissXD22200,22200.0,True,es un juego miy bueno y es en 1ra parsona :D,pipekissXD
22609,Zeno Clash,23.0,180,ClockworkLunatic22200,22200.0,True,WHAT THE OH MY GOD I DONT UNDERSTAND WHY IS WH...,ClockworkLunatic
36494,Zeno Clash,294.0,331,vschiffer22200,22200.0,True,An abstract yet brilliant journey in a world r...,vschiffer


In [88]:
file_name_to_read = './dataset_ready/steam_games_ver2.csv' # to load steam_games file to join next
steam_games4 = pd.read_csv(file_name_to_read)

In [89]:
merged_df3 = merged_df.merge(steam_games4, on='item_id') # merge items+reviews with steam_games --> one table!

In [22]:
merged_df3.head()

,item_name,playtime_forever,items_count,joined_id,item_id,recommend,review,user_id,publisher,genres,title,release_date,price,developer
0,Zeno Clash,271.0,277,7656119797098247922200,22200.0,True,It's unique and worth a playthrough.,76561197970982479,ACE Team,Action,Zeno Clash,2009-04-21,9.99,ACE Team
1,Zeno Clash,240.0,942,seantheextraprawnsheepguy22200,22200.0,True,"Combine FPS and Fighting games, mixed with the...",seantheextraprawnsheepguy,ACE Team,Action,Zeno Clash,2009-04-21,9.99,ACE Team
2,Zeno Clash,690.0,61,pipekissXD22200,22200.0,True,es un juego miy bueno y es en 1ra parsona :D,pipekissXD,ACE Team,Action,Zeno Clash,2009-04-21,9.99,ACE Team
3,Zeno Clash,23.0,180,ClockworkLunatic22200,22200.0,True,WHAT THE OH MY GOD I DONT UNDERSTAND WHY IS WH...,ClockworkLunatic,ACE Team,Action,Zeno Clash,2009-04-21,9.99,ACE Team
4,Zeno Clash,294.0,331,vschiffer22200,22200.0,True,An abstract yet brilliant journey in a world r...,vschiffer,ACE Team,Action,Zeno Clash,2009-04-21,9.99,ACE Team


In [16]:

# change dates with othes format: Jun 2015, Jul 2016, Feb 2011, Jull 2014
# Mar 2010, Oct 2016, Jan 2015, May 2014
merged_df3['release_date'] = merged_df3['release_date'].replace([''], '2014-07-01')

In [19]:
# change dates in blanks with 2016-07-01
merged_df3['release_date'] = merged_df3['release_date'].fillna('2016-07-01')

In [32]:
#change data type on column release_date to YYYY-MM-DD

merged_df3['release_date'] = pd.to_datetime(merged_df3['release_date'])

In [33]:
# create a colum year

merged_df3['year'] = merged_df3['release_date'].dt.year

In [26]:
# change the data type of column developer
merged_df3['developer'] = merged_df3['developer'].astype('str')

In [28]:
# apply the strip method to eliminate spaces
merged_df3['developer'] = merged_df3['developer'].str.strip()

In [50]:
# set o price in column price for free content Free to Play, Free to Play,Free, Free Mod, Free HITMAN™ Holiday Pack,Free Movie,
#Play the Demo, Install Now, Third-party, Free to Use, Play for Free!, Play WARMACHINE: Tactics Demo, blanks

merged_df3['price'] = merged_df3['price'].fillna('0')

merged_df3['price2'] = merged_df3['price'].replace({'Free to Play': '0','Free To Play': '0', 'Free': '0', 'Free Mod': '0', 'Free HITMAN™ Holiday Pack':'0', 'Free Movie':'0','Play the Demo':'0','Install Now':'0','Third-party':'0','Free to Use':'0','Play for Free!':'0','Play WARMACHINE: Tactics Demo':'0'})

#convert the type of price2 column to float with 2 decimal places
merged_df3['price2'] = merged_df3['price2'].astype(float).round(2)

In [ ]:
# function to make the sentiment analyst
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment < -0.1:
        return 0
    elif sentiment > 0.1:
        return 2
    else:
        return 1

In [ ]:
# change the data type of review column
merged_df3['review']=merged_df3['review'].astype(str)

In [ ]:
# create sentiment_analyst column
merged_df3['sentiment_analysis'] = merged_df3['review'].apply(get_sentiment)

In [52]:
# Export DataFrame to csv file
file_name = './dataset_ready/merged_users_items_reviews_games.csv' #file name included extension
pd.DataFrame(merged_df3, columns = merged_df3.columns).to_csv(file_name, index=False)  # "index=False" to avoid indexs columns

In [51]:
# Export DataFrame to csv file
file_name = './dataset_ready/merged_users_items_reviews_games.xlsx' #file name included extension
pd.DataFrame(merged_df3, columns = merged_df3.columns).to_excel(file_name, index=False)  # "index=False" to avoid indexs columns

In [ ]:

#set data ready for the use of the procedure by getting the item_id
data = pd.read_csv('./dataset_ready/merged_users_items_reviews_games.csv')
steam_merged = pd.read_csv('./dataset_ready/merged_users_items_reviews_games.csv')

#take out data not useful for tgis process
steam_merged.drop(['joined_id','user_id','genres','recommend','price','price2','playtime_forever','developer','release_date','year'],axis=1,inplace=True)
data.drop(['joined_id','user_id','genres','recommend','price','price2','playtime_forever','year'],axis=1,inplace=True)

data['item_id'] = data['item_id'].astype(str)
data['release_date'] = data['release_date'].astype(str)
data['title'] = data['title'].astype(str)
data['developer'] = data['developer'].astype(str)

#take put data duplicated on columns
data = data.drop_duplicates(subset='item_id', keep='first')
steam_merged = steam_merged.drop_duplicates(subset='item_id', keep='first')

# link needed data in one column
data['features'] = data[['title', 'developer', 'release_date']].apply(lambda x: ', '.join(x), axis=1)

data.drop(['title','developer','release_date'],axis=1,inplace=True)

# save final data frames to use in the query of genres
data.to_csv('./dataset_ready/games_steam.csv', index=False)
steam_merged.to_csv('./dataset_ready/games_id.csv', index=False)